In [6]:
import sys
sys.path.append('../../../')
sys.path.append('/home/sheida.rahnamai/GIT/HDN/')
import torch
import pickle
import numpy as np
from tqdm import tqdm
from lib.dataloader import CustomDataset, CombinedBatchSampler
import tifffile as tiff
from sklearn.utils import shuffle
import os
from boilerplate import boilerplate
import random
from skimage.measure import regionprops

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

patch_size = 64
sample_size = 350
centre_size = 4
n_channel = 32
hierarchy_level = 3
pad_size = (patch_size - centre_size) // 2

In [7]:
classes = ['uncategorized', 'nucleus', 'granule', 'mitochondria']
train_labeled_indices = []
val_labeled_indices = []
for cls in classes:
    with open(f'/group/jug/Sheida/pancreatic beta cells/download/train/10_percent_{cls}.pickle', 'rb') as file:
        train_labeled_indices.extend(pickle.load(file))
    with open(f'/group/jug/Sheida/pancreatic beta cells/download/val/10_percent_{cls}.pickle', 'rb') as file:
        val_labeled_indices.extend(pickle.load(file))

In [8]:
print(len(train_labeled_indices), len(val_labeled_indices))

10381 1177


In [9]:
dir = '/group/jug/Sheida/pancreatic beta cells/download/train/1_percent_mitochondria.pickle'
dir_save = '/group/jug/Sheida/pancreatic beta cells/download/train/0.1_percent_mitochondria.pickle'
with open(dir, 'rb') as file:
        x = pickle.load(file)
y = random.sample(x, len(x)//10)
with open(dir_save, 'wb') as file:
    pickle.dump(y, file)

In [10]:
dir = '/group/jug/Sheida/pancreatic beta cells/download/train/1_percent_mitochondria.pickle'
with open(dir, 'rb') as file:
        x = pickle.load(file)
print(len(x))

73


In [3]:
# train data

data_dir = "/group/jug/Sheida/pancreatic beta cells/download/"
keys = ['high_c1', 'high_c2', 'high_c3']

# Load source images
train_img_paths = [os.path.join(data_dir + 'train/' + key + f"/{key}_source.tif") for key in keys]
train_lbl_paths = [os.path.join(data_dir + 'train/' + key + f"/{key}_gt.tif") for key in keys]
val_img_paths = [os.path.join(data_dir + 'val/' + key + f"/{key}_source.tif") for key in keys]
val_lbl_paths = [os.path.join(data_dir + 'val/' + key + f"/{key}_gt.tif") for key in keys]

train_images = {key: tiff.imread(path) for key, path in zip(keys, train_img_paths)}
train_labels = {key: tiff.imread(path) for key, path in zip(keys, train_lbl_paths)}

val_images = {key: tiff.imread(path) for key, path in zip(keys, val_img_paths)}
val_labels = {key: tiff.imread(path) for key, path in zip(keys, val_lbl_paths)}

for key in tqdm(keys, desc='filtering out outside of the cell'):
   filtered_image, filtered_label = boilerplate._filter_slices(train_images[key], train_labels[key])
   train_images[key] = filtered_image
   train_labels[key] = filtered_label

   filtered_image, filtered_label = boilerplate._filter_slices(val_images[key], val_labels[key])
   
   val_images[key] = filtered_image
   val_labels[key] = filtered_label

# compute mean and std of the data
all_elements = np.concatenate([train_images[key].flatten() for key in keys])
data_mean = np.mean(all_elements)
data_std = np.std(all_elements)

# normalizing the data
for key in tqdm(keys, 'Normalizing data'):
   train_images[key] = (train_images[key] - data_mean) / data_std
   val_images[key] = (val_images[key] - data_mean) / data_std


Normalizing data: 100%|██████████| 3/3 [00:02<00:00,  1.30it/s]


In [5]:
train_set = CustomDataset(train_images, train_labels)
val_set = CustomDataset(val_imvages, val_labels)

Extracting patches from high_c1: 0it [00:00, ?it/s]


NameError: name 'regionprops' is not defined

In [ ]:
print(f"train set size: {len(train_set)}, val set size: {len(val_set)}")
print(f"Number of given GT labels in train data: {len(train_labeled_indices)}, Number of given GT labels in val data: {len(val_labeled_indices)}")

train set size: 103821, val set size: 11781
Number of given GT labels in train data: 10381, Number of given GT labels in val data: 1177
